In [2]:
import pandas as pd
import numpy as np
from scipy.stats import norm

In [4]:
#WYSCOUT DATA

#importing your excel file
df = pd.read_csv('attackers_eliteserien.csv') 

In [6]:
df.head()

,Unnamed: 0,Player,Age,League,Position,Team,Birth country,Non-penalty goals per 90,npxG per 90,"Successful dribbles, %",...,"Accurate passes, %",Shots per 90,"Accurate crosses, %",Smart passes per 90,xA per Shot Assist,Accelerations per 90,Aerial duels won per 90,Fouls suffered per 90,npxG per shot,Crosses per 90
0,0,J. Reitan-Sunde,18,Eliteserien 2024,"LWF, RWF",Rosenborg,Norway,0.237288,0.186441,0.135593,...,0.542373,0.050847,0.508475,0.262712,0.288136,0.847458,0.177966,0.847458,0.711864,0.779661
1,1,F. Tewelde,18,Eliteserien 2024,"CF, LW, LWF",Odds,Norway,0.703390,0.576271,0.389831,...,0.169492,0.677966,0.864407,0.559322,0.372881,0.983051,0.593220,0.661017,0.372881,0.559322
2,2,B. Bang-Kittilsen,19,Eliteserien 2024,"RWF, RW, LAMF",Odds,Norway,0.661017,0.440678,0.016949,...,0.203390,0.161017,0.067797,0.127119,0.983051,0.245763,0.610169,0.864407,0.796610,0.271186
3,3,M. Broholm,19,Eliteserien 2024,"RWF, CF",Rosenborg,Norway,0.601695,0.203390,0.644068,...,0.847458,0.796610,0.610169,0.813559,0.610169,0.796610,0.050847,0.661017,0.067797,0.864407
4,4,H. Mikaelsson,20,Eliteserien 2024,CF,Kristiansund,Iceland,0.084746,0.338983,0.864407,...,0.474576,0.508475,0.372881,0.127119,0.338983,0.423729,0.881356,0.432203,0.237288,0.245763


In [8]:
df.columns

Index(['Unnamed: 0', 'Player', 'Age', 'League', 'Position', 'Team',
       'Birth country', 'Non-penalty goals per 90', 'npxG per 90',
       'Successful dribbles, %', 'Goal conversion, %', 'Touches in box per 90',
       'Accurate short / medium passes, %', 'Passes per 90',
       'Shot assists per 90', 'xA per 90', 'Assists per 90',
       'Second assists per 90', 'Third assists per 90',
       'Progressive passes per 90', 'Progressive runs per 90', 'Duels won, %',
       'pAdj Tkl+Int per 90', 'Successful defensive actions per 90',
       'PAdj Sliding tackles', 'Defensive duels won, %', 'Fouls per 90',
       'Cards per 90', 'Shots blocked per 90', 'PAdj Interceptions',
       'Aerial duels won, %', 'Accurate long passes, %',
       '1st, 2nd, 3rd assists', 'Conceded goals per 90',
       'Prevented goals per 90', 'Shots against per 90', 'Save rate, %',
       'Exits per 90', 'Passes', 'Long Pass\nCmp %',
       '% of Passes\nBeing Short', '% of Passes\nBeing Lateral',
       'Rece

In [10]:
#Optional: select your tier
#df = df[df['Tier'] =='Tier 2']

#Select your league
#df = df[df['League'] == '2. Bundesliga'] 

#Select your position in the Wyscout data - if they only played one position
df = df[df['Position'] =='CF']

#Select this if you want to find all players that have played CF
df = df[df['Position'].str.contains('CF')]

#Select the minuted played
#df = df[df['Minutes played'] > 900]

#Select the age
df = df[df['Age'] <= 25]

In [12]:
df.columns

Index(['Unnamed: 0', 'Player', 'Age', 'League', 'Position', 'Team',
       'Birth country', 'Non-penalty goals per 90', 'npxG per 90',
       'Successful dribbles, %', 'Goal conversion, %', 'Touches in box per 90',
       'Accurate short / medium passes, %', 'Passes per 90',
       'Shot assists per 90', 'xA per 90', 'Assists per 90',
       'Second assists per 90', 'Third assists per 90',
       'Progressive passes per 90', 'Progressive runs per 90', 'Duels won, %',
       'pAdj Tkl+Int per 90', 'Successful defensive actions per 90',
       'PAdj Sliding tackles', 'Defensive duels won, %', 'Fouls per 90',
       'Cards per 90', 'Shots blocked per 90', 'PAdj Interceptions',
       'Aerial duels won, %', 'Accurate long passes, %',
       '1st, 2nd, 3rd assists', 'Conceded goals per 90',
       'Prevented goals per 90', 'Shots against per 90', 'Save rate, %',
       'Exits per 90', 'Passes', 'Long Pass\nCmp %',
       '% of Passes\nBeing Short', '% of Passes\nBeing Lateral',
       'Rece

In [14]:
# Goalscoring strikers
original_metrics = ["npxG per 90", "Goal conversion, %", "Received\nPasses",
               "Shot assists per 90", "xA per 90", "Shots per 90",
              "Aerial duels won, %", "Touches in box per 90", "Non-penalty goals per 90"]

weights = [5, 5, 3,
           1, 1, 1,
           0.5, 3, 1]

In [18]:
# Calculate the composite score for the original metrics
df["Goalscoring strikers"] = np.dot(df[original_metrics], weights)

# Calculate the mean and standard deviation for the composite score of the original metrics
original_mean = df["Goalscoring strikers"].mean()
original_std = df["Goalscoring strikers"].std()

# Calculate the z-scores for the composite score of the original metrics
df["Goalscoring strikers"] = (df["Goalscoring strikers"] - original_mean) / original_std

# Map the z-scores of the original metrics to a range of 0 to 100 with two decimal places
df["Goalscoring strikers(0-100)"] = (norm.cdf(df["Goalscoring strikers"]) * 100).round(2)

In [34]:
# Sort the DataFrame by the z-scores of the original metrics in ascending order
df_original = df.sort_values("Goalscoring strikers", ascending=False)

In [36]:
df_original.head()

,Unnamed: 0,Player,Age,League,Position,Team,Birth country,Non-penalty goals per 90,npxG per 90,"Successful dribbles, %",...,"Accurate crosses, %",Smart passes per 90,xA per Shot Assist,Accelerations per 90,Aerial duels won per 90,Fouls suffered per 90,npxG per shot,Crosses per 90,Goalscoring strikers,Goalscoring strikers(0-100)
22,22,K. Høgh,23,Eliteserien 2024,CF,Bodø / Glimt,Denmark,1.000000,0.949153,0.542373,...,0.923729,0.338983,0.762712,0.245763,0.864407,0.500000,0.983051,0.033898,1.021312,84.64
21,21,N. Holm,23,Eliteserien 2024,CF,Rosenborg,Norway,0.830508,0.915254,0.788136,...,0.067797,0.508475,1.000000,0.101695,0.762712,0.152542,1.000000,0.135593,0.996999,84.06
5,5,H. Meister,20,Eliteserien 2024,CF,Sarpsborg 08,Denmark,0.915254,0.847458,0.186441,...,0.203390,0.127119,0.898305,0.686441,0.813559,0.169492,0.864407,0.169492,0.778179,78.18
18,18,L. Nordås,22,Eliteserien 2024,CF,Tromsø,Norway,0.813559,0.898305,0.610169,...,0.322034,0.127119,0.932203,0.500000,0.830508,0.432203,0.881356,0.296610,0.671200,74.90
15,15,E. Melkersen,21,Eliteserien 2024,CF,Strømsgodset,Norway,0.762712,0.796610,0.220339,...,0.372881,0.372881,0.152542,0.177966,0.711864,0.372881,0.779661,0.076271,-0.256354,39.88


In [30]:
# Print the list of player names, squads, competitions, and the z-scores
print(df[["Player", "Age", "Team", "Goalscoring strikers(0-100)"]])

           Player  Age          Team  Goalscoring strikers(0-100)
4   H. Mikaelsson   20  Kristiansund                         6.04
5      H. Meister   20  Sarpsborg 08                        78.18
15   E. Melkersen   21  Strømsgodset                        39.88
18      L. Nordås   22        Tromsø                        74.90
19        K. Lien   23  Kristiansund                        15.89
21        N. Holm   23     Rosenborg                        84.06
22        K. Høgh   23  Bodø / Glimt                        84.64
28      S. Diarra   24     Haugesund                        25.43


In [32]:
df_sorted = df[["Player", "Age", "Team", "Goalscoring strikers(0-100)"]].sort_values(
    by="Goalscoring strikers(0-100)", ascending=False
)
df_sorted.head(10)


,Player,Age,Team,Goalscoring strikers(0-100)
22,K. Høgh,23,Bodø / Glimt,84.64
21,N. Holm,23,Rosenborg,84.06
5,H. Meister,20,Sarpsborg 08,78.18
18,L. Nordås,22,Tromsø,74.90
15,E. Melkersen,21,Strømsgodset,39.88
28,S. Diarra,24,Haugesund,25.43
19,K. Lien,23,Kristiansund,15.89
4,H. Mikaelsson,20,Kristiansund,6.04
